>[组合键的阿斯克码表](https://blog.csdn.net/iodoo/article/details/49175749)<br>
>[expect教程](http://xstarcd.github.io/wiki/shell/expect.html)

## expect脚本的执行
脚本必须具有可执行权限，使用./\script_name.sh执行 或者 expect script_name.sh

## expect简单用法
### 基本语句
* #!/usr/bin/expect<br>
    告诉操作系统使用哪个shell来执行本代码。expect与bash是一样的。
    
* set timeout 30<br>
    设置超时时间，单位：s
    
* spawn ssh -l username 192.168.1.1<br>
    spawn ./t.sh<br>
    expect的内部命令，用于启动一个进程。
    
* expect "strings"<br>
    expect的内部命令。判断上次输出结果是否包含字符串“strings”，如果有则立即返回，否则等待一段时间后返回(该时间是set timeout设定的)。
    
* interact<br>
    执行完成后保持交互状态，把控制权交给控制台，进行手工操作。没有这句的话，spawn启动的程序将会立即停止。
    
        1 # ex.sh
        2 #!/usr/bin/expect
        3 set timeout 1
        4 spawn ./t.sh
        5 expect "start"
        6 send "yfs\n"
        7 expect "asdfasdf"
        8 send "男\n"
        9 #interact
    
      1 # t.sh
      2 #!/bin/bash
      3 echo "这是shell程序：start"
      4 
      5 read name
      6 read sex
      7 
      8 echo $name, $sex
      9 echo "这是shell程序：end"
      10 
      11 read var
      12 echo $var


* send:
    向spawn启动的进程p发送其待输入的东西。
    
* send_user:
    回显用户发出的消息。类似shell中的echo。
    
* 获取命令行参数： <br>
    $argc: 参数个数<br>
    [lindex $argv 0]: 第一个参数 <br>
    [lindex $argv 2]: 第三个参数<br>
    [lrange $argv 0 2]: 第1个到第3个参数<br>

* 读取用户输入<br>
expect_user -re "(.\*)\n"<br>表示正则表达式匹配用户按下回车前输入的所有字符<br><br>
send_user "$expect_out(1, string)\n" <br>表示第一个匹配的内容，即回车前所有字符<br><br>
exp_continue:<br>
表示循环

In [7]:
# 例1
!cat -n shell_prog/read.sh

     1	#!/usr/bin/expect
     2	
     3	set timeout 3
     4	expect_user {
     5	    -re hello {
     6	        puts 'Ihello'
     7	        exp_continue
     8	    }
     9	    -re world {
    10	        puts 'Iworld'
    11	    }
    12	    default {
    13	        puts 'default'
    14	    }
    15	}
    16	expect eof

<font color='ff0000' size=5>未解之谜：</font>程序第9行并不能正确匹配

In [9]:
# 例2
!cat -n shell_prog/read2.sh

     1	#!/usr/bin/expect
     2	set timeout 3
     3	expect_user {
     4	    -re ^abc.* {
     5	        puts 'Ihello'
     6	        exp_continue
     7	    }
     8	
     9	    -re ^a.*b$ {
    10	        puts 'Iworld'
    11	    }
    12	
    13	    default {
    14	        puts 'default'
    15	    }
    16	}
    17	
    18	expect eof

`expect_user -re '^a.*' {puts "Input:a"}`是错误的<br>
应改为：
`#!/usr/bin/expect
set timeout 3
expect_user {
	-re hello { 
		puts "Input:a" 
	}
}
`<br>
<font color='#aa55ff' size=5>注意：-re hello，中hello不能加引号</font>

### 变量赋值，if判断句：
    set 变量名 值或变量：后面的赋值给前面的<br>
puts：打印字符串类似echo<br>
**if判断句：**<br>
    ‘{’的左边要有空格<br>
注意：puts参数中不能有空格，除非用引号引起来，否则报错。

In [23]:
cat shell_prog/a.sh

#!/usr/bin/expect

set timeout 1
set arg [lindex $argv 0]
#set t 9
#set arg $t

if { $arg>0 } {
    puts '参数是正数',$arg
} elseif { $arg == 0 } {
    puts '参数为0',$arg
} else {
    puts '参数是负数',$arg
}
expect eof


In [22]:
!expect shell_prog/a.sh 9

In [24]:
!expect shell_prog/a.sh 0

In [25]:
!expect shell_prog/a.sh -9

### 字符串的比较：

In [36]:
cat shell_prog/b.sh

#!/usr/bin/expect

set timeout 1
set arg [lindex $argv 0]

if { $arg == "apple" } {
    puts '这是一个苹果'
} elseif { $arg == "banana" } {
    puts '这是一个香蕉'
} else {
    puts '不知道这是什么东西'
}
    

In [33]:
!expect shell_prog/b.sh 'apple'

In [34]:
!expect shell_prog/b.sh 

In [35]:
!expect shell_prog/b.sh 'banana'

### switch语句

<font color='#ff00cc' size=5>注意{左边要有空格</font>

In [39]:
cat shell_prog/switch.sh

#!/usr/bin/expect

set timeout 1
set arg [lindex $argv 0]

switch $arg {
    "apple" { puts "this is an apple!"}
    "banana" { pust "this is a banana"}
    default { puts "other"}
}

In [38]:
!expect shell_prog/switch.sh 'apple'

## 其他

#### {}大括号

1. 保留原有字符串，如`set ver {a$b[set c 3]\tddd}`将{}中的一坨东西赋值给var

2. 续行
    
    if { $count < 0} { # 正确
        break;
    }
    
    
    if { $count < 0} # 错误 
    { 
        break;
    }

#### []：执行命令

In [2]:
cat -n shell_prog/c.sh

     1	#!/usr/bin/expect
     2	
     3	set timeout 3
     4	set a [set b 6]
     5	puts a=$a
     6	puts b=$b

In [3]:
!expect shell_prog/c.sh

# expect_out(buffer)与expect_out(0,string)

<font color='#ff5600' size=3>
    \$expect_out(buffer):存储了所有对expect的输入<br>
    \$expecct_out(0,string): 存储匹配到expect参数的输入
</font>

In [19]:
!bash shell_prog/launch.sh "1.sh" "no"

     1	#!/usr/bin/expect
     2	expect "hi\n"
     3	send "you typed <$expect_out(buffer)>"
     4	send "but I only expected <$expect_out(0,string)>"


<img src='images/1.png'/>

In [22]:
# 多分支模式
!bash shell_prog/launch.sh "2.sh" "no"

     1	#!/usr/bin/expect
     2	set timeout 6
     3	expect {
     4	    "hi" {puts "input hi"; exp_continue}
     5	    "hello" {puts "input hello"; exp_continue}
     6	    default {puts default}
     7	}

<img src='images/3.png'/>

In [23]:
# 单分支模式
!bash shell_prog/launch.sh "3.sh" "no"

     1	#!/usr/bin/expect
     2	set timeout 5
     3	expect "hi" {puts "input hi"}

<img src='images/4.png'/>


[tk](https://www.ibm.com/developerworks/cn/education/linux/l-tcl/l-tcl-blt.html)